In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_probability as tfp
import arviz as az

import IPython

from meridian import constants
from meridian.data import load
from meridian.data import test_utils
from meridian.model import model
from meridian.model import spec
from meridian.model import prior_distribution
from meridian.analysis import optimizer
from meridian.analysis import analyzer
from meridian.analysis import visualizer
from meridian.analysis import summarizer
from meridian.analysis import formatter

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
# If utils exists with the needed encoded functions
from utils import *
# check if GPU is available
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\\n'.format(ram_gb))
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
print("Num CPUs Available: ", len(tf.config.experimental.list_physical_devices('CPU')))


In [ ]:
['Unnamed: 0',
 'Starting week',
 'Year',
 "UK L'Oreal Paris Haircare Offline Average Price (in pound)",
 "UK L'Oreal Paris Haircare Online Average Price (in pound)",
 "UK L'Oreal Paris Haircare Total Weigheted Promotion Distribution (%)",
 "UK L'Oreal Paris Haircare Total Offline Sellout Value (in pound)",
 "UK L'Oreal Paris Haircare Total Offline Sellout Units",
 "UK L'Oreal Paris Haircare Total Online Sellout Value (in pound)",
 "UK L'Oreal Paris Haircare Total Online Sellout Units",
 'Starting week.1',
 'impressions_digital_tv',
 'impressions_online_multiformat_ads_transaction',
 'impressions_online_video_content_platforms',
 'impressions_paid_search_awarenessconsideration',
 'impressions_paid_search_transaction',
 'impressions_social_media_awarenessconsideration',
 'impressions_social_media_transaction',
 'spend_digital_tv',
 'spend_online_multiformat_ads_transaction',
 'spend_online_video_content_platforms',
 'spend_paid_search_awarenessconsideration',
 'spend_paid_search_transaction',
 'spend_social_media_awarenessconsideration',
 'spend_social_media_transaction',
 'Starting week.2',
 'engagement_influencer_management',
 'spend_influencer_management',
 'Starting week.3',
 'grp_traditional_tv',
 'spend_traditional_tv']

In [ ]:
coord_to_columns = load.CoordToColumns(
    time='Starting week',
    # geo='geo',
    # population='population',
    kpi='recrutements_hebdo', # choose the target variable you want
    revenu_per_kpi= "UK L'Oreal Paris Haircare Total Online Sellout Units",
    controls= [
        "UK L'Oreal Paris Haircare Offline Average Price (in pound)",
        "UK L'Oreal Paris Haircare Online Average Price (in pound)",
        "UK L'Oreal Paris Haircare Total Weigheted Promotion Distribution (%)"
                ],
    media= [
        'impressions_digital_tv',
        'impressions_online_multiformat_ads_transaction',
        'impressions_online_video_content_platforms',
        'impressions_paid_search_awarenessconsideration',
        'impressions_paid_search_transaction',
        'impressions_social_media_awarenessconsideration',
        'impressions_social_media_transaction',
        ],
    media_spend=[
        'spend_digital_tv',
        'spend_online_multiformat_ads_transaction',
        'spend_online_video_content_platforms',
        'spend_paid_search_awarenessconsideration',
        'spend_paid_search_transaction',
        'spend_social_media_awarenessconsideration',
        'spend_social_media_transaction',
                ],
    reach=['reach_tvfixe',
                'reach_tvmobile',
                'reach_ooh'],
    frequency=['frequency_tvfixe',
                    'frequency_tvmobile',
                    'frequency_ooh']
    rf_spend=['spend_tv_fixe',
                'spend_tv_mobile',
                'spend_ooh'],
)

In [ ]:
media_to_channel={
    'impressions_digital_tv' : 'digital_tv',
    'impressions_online_multiformat_ads_transaction': "multiformat_ads",
    'impressions_online_video_content_platforms': "video_content_platforms",
    'impressions_paid_search_awarenessconsideration': "paid_search_awareness_consideration",
    'impressions_paid_search_transaction': "search_transaction",
    'impressions_social_media_awarenessconsideration':"social_media_awareness_consideration",
    'impressions_social_media_transaction': "social_media_transaction",
}
media_spend_to_channel={
    'spend_digital_tv': 'digital_tv',
    'spend_online_multiformat_ads_transaction': "multiformat_ads",
    'spend_online_video_content_platforms': "video_content_platforms",
    'spend_paid_search_awarenessconsideration': "paid_search_awareness_condiseration",
    'spend_paid_search_transaction': "search_transactino",
    'spend_social_media_awarenessconsideration': "social_media_awareness_consideration",
    'spend_social_media_transaction': "social_media_transaction",
}
reach_to_channel={
            'reach_tv_fixe':'tv_fixe',
            'reach_tv_mobile':'tv_mobile',
            'reach_ooh':'ooh'
            }
frequency_to_channel={
            'frequency_tv_fixe':'tv_fixe',
            'frequency_tv_mobile':'tv_mobile',
            'frequency_ooh':'ooh'
    }
rf_spend_to_channel={
            'spend_tv_fixe':'tv_fixe',
    'spend_tv_mobile':'tv_mobile',
            'spend_ooh':'ooh'
    }

loader = load.DataFrameDataLoader(
    df=df, # Attention, le csv doit être au même endroit que le script python
    kpi_type='non_revenue',
    coord_to_columns=coord_to_columns,
    media_to_channel=media_to_channel,
    media_spend_to_channel=media_spend_to_channel,
    reach_to_channel=reach_to_channel,
    frequency_to_channel= frequency_to_channel,
    rf_spend_to_channel= rf_spend_to_channel,
)

In [ ]:
# Latence et saturation des canaux média  |  Meridian  |  Google for Developers
alpha_prior = tfp.distributions.Uniform(
low= [0.0001, 0.0001, 0.0001, 0.0001, 0.0001,0.0001],
high=[0.4, 0.3, 0.3, 0.6, 0.3, 0.4],
name=constants.ALPHA_M
)

# Elasticité cumulée, ie expected per euro spent during the period
ec_priors = tfp.distributions.TruncatedNormal(
    loc= [0.1, 0.2, 0.2,0.3, 0.4, 0.2],
    scale=[2.0, 2.0, 2.0, 2.0, 2.0, 2.0],
    low = 0.001,
    high= 10.0,
    name=constants.EC_M
)

# Intensity of media effect
# Google → strong direct effet, TV → harder to measure its effect through time
beta_prior = tfp.distributions.LogNormal(
    loc= [0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
scale=[0.17, 0.2, 0.2, 0.13, 0.17, 0.17]
, # pour jouer sur l'incertitude
    name=constants.BETA_M
)

alpha_prior_rf = tfp.distributions.Uniform(
low= [0.5, 0.5, 0.001],
high=[1.0, 1.0, 0.7],
name=constants.ALPHA_M
)

ec_priors_rf = tfp.distributions.TruncatedNormal(
    loc= [0.4, 0.4, 0.3],
    scale=[2.0, 2.0, 2.0],
    low = 0.001,
    high= 10.0,
    name=constants.EC_M
)

beta_prior_rf = tfp.distributions.LogNormal(
    loc= [0.0, 0.0, 0.0],
scale=[0.095, 0.09, 0.11]
, # pour jouer sur l'incertitude
    name=constants.BETA_M
)

knot_prior = tfp.distributions.Normal(
    loc = 0.0,
    scale = 0.2,
    name=constants.KNOT_VALUES)
# How much the baseline of each geo is allowed to changed compared to the one with the most population
baseline_prior = tfp.distributions.Normal(
		loc = 0.0,
		scale = 0.01,
		name = constants.BASELINE_GEO
)
prior = prior_distribution.PriorDistribution(
    knot_values=knot_prior,
    tau_g_excl_baseline=baseline_prior,
    alpha_m=alpha_prior,
    alpha_rf = alpha_prior_rf,
    beta_m=beta_prior,
    beta_rf = beta_prior_rf,
    ec_m=ec_priors,
    ec_rf = ec_priors_rf
)

In [ ]:
model_spec = spec.ModelSpec(knots = 20,
														prior=prior,
                            media_prior_type='coefficient',
                            rf_prior_type = 'coefficient',
                            baseline_geo = None,
                            max_lag= 18)

mmm =model.Meridian(
    input_data=df,
    model_spec=model_spec
)

mmm.sample_prior(500)
mmm.sample_posterior(n_chains=10, n_adapt=2000, n_burnin=500, n_keep=1000, seed=1)